# Distributed Homomorphic Public Key Encryption

In [131]:
from math import gcd
from random import randint, choice, shuffle
from sympy import discrete_log
from hashlib import sha256
n = 16

## Generate group Z*p

In [132]:
def is_prime(n: int) -> bool:
    for i in range(2, n // 2):
        if n % i == 0:
            return False
    return True

def hash_to_group(tag: str, p: int, q: int) -> int: # ChatGPT
    tag = tag.encode()
    h = int(sha256(tag).hexdigest(), 16) % p
    return pow(h, (p-1)//q, p)

assert is_prime(17)
assert is_prime(47)
assert not is_prime(16)

In [133]:
class Group:
    def __generate(self):
        p, g = self.p, self.g
        l = []
        cur = g
        while cur != 1:
            l.append(cur)
            cur = (cur * g) % p
        l.append(1)
        self.q = len(l)
        self.group = tuple(l)

    def __init__(self, p, g):
        assert is_prime(p), f"Value for p {p} is not a prime"
        assert g < p, f"Equation g < p is not satisfied when g = {g} and p = {p}"
        self.p = p
        self.g = g
        self.__generate()

    def __str__(self):
        return f"(p: {self.p}, g: {self.g}, q: {self.q}): G = {self.group}"

    def getValues(self):
        return self.p, self.g, self.q, self.group

    def getP(self):
        return self.p

    def getG(self):
        return self.g

    def getQ(self):
        return self.q

    def getPGQ(self):
        return self.p, self.g, self.q

    def getGroup(self):
        return self.group
    
    def isMember(self, x: int):
        return x in self.group
    
    def getInverse(self, x: int):
        p = self.getP()
        return pow(x, p - 2, p)
    
    def getRandomGroupFromSameOrder(self):
        p, q, g = self.p, self.q, self.g
        r = randint(1, q - 1)
        while gcd(q, r) != 1:
            r = randint(1, q - 1)
        gp = pow(g, r, p)
        newGroup = Group(p, gp)
        assert self.getQ() == newGroup.getQ(), f"{self.getQ()} vs {newGroup.getQ()}"
        return newGroup

group = Group(47, 17)
print(group)
assert group.getGroup() == (17, 7, 25, 2, 34, 14, 3, 4, 21, 28, 6, 8, 42, 9, 12, 16, 37, 18, 24, 32, 27, 36, 1)
assert group.isMember(17)
assert group.isMember(1)
assert group.isMember(42)
assert not group.isMember(15)
for each in group.getGroup():
    assert (each * group.getInverse(each)) % group.getP() == 1

(p: 47, g: 17, q: 23): G = (17, 7, 25, 2, 34, 14, 3, 4, 21, 28, 6, 8, 42, 9, 12, 16, 37, 18, 24, 32, 27, 36, 1)


In [134]:
for i in range(int(10e3), 0, -1):
    for j in range(2, i // 2, 1):
        if i % j == 0:
            break
    if i % j != 0:
        print("Test", i)
        q = i
        assert is_prime(q)
        k = 4
        p = k * q + 1

        if is_prime(p):
            g = pow(randint(0, p - 1), k, p)
            break

print("Done", p, g, q)
group = Group(p, g)
# print(group)

set = []
for each in group.getGroup():
    assert each not in set
    set.append(each)

Test 9973
Test 9967
Done 39869 9682 9967


# Divertible Schnorr Proof

Create a signature of my secret to allow someone else who doesn't know my secret to prove at my place that I know the secret.
In this world, we add a third agent that is an Issuer responsible to provide a prover with a signature to make later a proof to the verifier.

In [135]:
def divertible_schnorr_hash(g: int, h: int, ap: int, m: str = "Il etait une fois un patate dans une friteuse...."):
    return g * h * ap * hash(m)

class DivertibleSchnorrIssuer:
    def __init__(self, group: Group):
        self.group = group
        p, g, q = group.getPGQ()
        self.x = randint(0, q - 1)
        self.h = pow(g, self.x, p)
    
    def getCommitment(self):
        p, g, q = group.getPGQ()
        r = randint(0, q - 1)
        a = pow(g, r, p)
        self.r = r
        return a, self.h
    
    def getProof(self, e: int):
        q = self.group.getQ()
        r = self.r
        x = self.x
        f = (r + e * x) % q
        self.f = f
        return f
    
    def getPublicKey(self):
        return self.h

class DivertibleSchnorrProver:
    def __init__(self, group: Group):
        self.group = group
    
    def getChallenge(self, a: int, h: int):
        self.a = a
        self.h = h
        p, g, q = group.getPGQ()

        w = randint(1, q - 1) # w in Zq*
        self.w = w
        w_1 = pow(w, q - 2, q) # Because w_1 builds e in Zq

        rp = randint(0, q - 1)
        self.rp = rp
        ap = pow(a, w, p) * pow(g, rp, p) % p
        self.ap = ap
        ep = divertible_schnorr_hash(g, h, ap) % q
        self.ep = ep

        e = (ep * w_1) % q
        self.e = e
        return e
    
    def acceptProof(self, f: int):
        rp = self.rp
        w = self.w
        fp = (rp + w * f) % q
        self.fp = fp

    def getSignature(self):
        return (self.ap, self.fp)
        
class DivertibleSchnorrVerifier:
    def __init__(self, group: Group):
        self.group = group
    
    def checkSignature(self, h: int, signature: tuple):
        ap, fp = signature
        p, g, q = group.getPGQ()
        ep = divertible_schnorr_hash(g, h, ap)
        return pow(g, fp, p) == (ap * pow(h, ep, p)) % p

issuer = DivertibleSchnorrIssuer(group)
prover = DivertibleSchnorrProver(group)
verifier = DivertibleSchnorrVerifier(group)
a, h = issuer.getCommitment()
e = prover.getChallenge(a, h)
f = issuer.getProof(e)
prover.acceptProof(f)
assert verifier.checkSignature(issuer.getPublicKey(), prover.getSignature())

In [136]:
def divertible_schnorr_signature_transcript():
    issuer = DivertibleSchnorrIssuer(group)
    while True:
        issuer2 = DivertibleSchnorrIssuer(group)
        if issuer.x != issuer2.x:
            break

    prover = DivertibleSchnorrProver(group)
    prover2 = DivertibleSchnorrProver(group)
    
    a, h = issuer.getCommitment()
    e = prover.getChallenge(a, h)
    f = issuer.getProof(e)
    prover.acceptProof(f)

    a, h = issuer2.getCommitment()
    e = prover2.getChallenge(a, h)
    f = issuer2.getProof(e)
    prover2.acceptProof(f)

    verifier = DivertibleSchnorrVerifier(group)

    assert verifier.checkSignature(issuer.getPublicKey(), prover.getSignature())
    assert not verifier.checkSignature(issuer2.getPublicKey(), prover.getSignature())
    assert verifier.checkSignature(issuer2.getPublicKey(), prover2.getSignature())
    assert not verifier.checkSignature(issuer.getPublicKey(), prover2.getSignature())

for _ in range(100):
    divertible_schnorr_signature_transcript() # It can rarely fail if by bad luck randomly the signatures becomes the same so we need a group large enough

# Commitment

In [137]:
class MultiPedersonCommitment:
    def __init__(self, group: Group, secrets: list):
        self.group = group
        self.secrets = secrets
        self.nonce = randint(0, self.group.getQ() - 1)
        self.__compute_generators()
        self.__compute_commitment()

    def __str__(self):
        return f"""
Commitment :
    r = {self.nonce}
    u = {self.generators}
    m = {self.secrets}
    c = {self.commitment}
        """

    def __compute_generators(self):
        p, g, q = self.group.getPGQ()
        n = len(self.secrets) + 1
        assert n <= q
        u = [0] * n
        Zq = [i for i in range(q)]
        shuffle(Zq)
        for i in range(n):
            u[i] = hash_to_group(f"u{i}", p, q)
        self.generators = u

    def __compute_commitment(self):
        p, g, q = self.group.getPGQ()
        r = self.nonce
        m, u = self.secrets, self.generators
        c = 1
        for i in range(len(m)):
            c *= pow(u[i + 1], m[i], p)
        c *= pow(u[0], r, p)
        c %= p
        self.commitment = c

    def getM(self):
        return self.secrets
    
    def getU(self):
        return self.generators
    
    def getR(self):
        return self.nonce
    
    def getC(self):
        return self.commitment
    
    def getNonce(self):
        return self.nonce

    def getSecrets(self):
        return self.secrets

    def getReRandomized(self):
        c = self.commitment
        u0 = self.generators[0]
        p, g, q = self.group.getPGQ()
        noncep = randint(0, q - 1)
        return (c * pow(u0, noncep, p)) % p, noncep

age = 22
credit = 147
noma = 14
commitment = MultiPedersonCommitment(group, [age, credit, noma])
print(commitment)
print(commitment.getReRandomized())


Commitment :
    r = 674
    u = [14527, 38781, 32038, 19192]
    m = [22, 147, 14]
    c = 21074
        
(13193, 1245)


In [138]:
def dumb_hash(elements: list):
    h = 1
    for each in elements:
        h *= each
    return h

class CommitmentIssuer:
    def __init__(self, group: Group):
        self.group = group
        p, g, q = group.getPGQ()
        self.x = randint(0, q - 1)
        self.h = pow(g, self.x, p)

    def getPublicKey(self):
        return self.h

    def getReCommitment(self, c: int, PI: tuple):
        p, g, q = self.group.getPGQ()
        A, F, u = PI
        left = 1
        e = dumb_hash(A) % q
        right = pow(c, e, p)
        for i in range(len(u)):
            left = left * pow(u[i], F[i], p) % p
            right = right * A[i] % p
        
        if left != right:
            return None
        
        x = self.x
        d = pow(c, x, p)
        v0 = pow(u[0], x, p)
        R = randint(0, q - 1)
        self.R = R
        a = pow(g, R, p)
        b = pow(c, R, p)
        a0 = pow(u[0], R, p)
        return d, a, b, a0, v0
    
    def getProof(self, e: int):
        R, x, q = self.R, self.x, self.group.getQ()
        f = (R + e * x) % q
        return f

class CommitmentProver:
    def __init__(self, group: Group, commitment: MultiPedersonCommitment):
        self.group = group
        self.commitment = commitment

    def getCommitment(self):
        p, g, q = self.group.getPGQ()
        c = self.commitment.getC()
        u = self.commitment.getU()
        r = [0] * len(u)
        A = [0] * len(u)
        for i in range(len(u)):
            r[i] = randint(0, q - 1)
            A[i] = pow(u[i], r[i], p)
        e = dumb_hash(A) % q
        F = [0] * len(u)
        nonce, secrets = self.commitment.getNonce(), self.commitment.getSecrets()
        F[0] = (r[0] + nonce * e) % q
        for i in range(1, len(u)):
            F[i] = (r[i] + secrets[i - 1] * e) % q
        self.r, self.A, self.F = r, A, F
        return c, (A, F, u)
    
    def getChallenge(self, h: int, d: int, a: int, b: int, a0: int, v0: int):
        p, g, q = self.group.getPGQ()
        rp = randint(0, q - 1)
        w = randint(1, q - 1)
        Rp = randint(0, q - 1)
        self.rp, self.w, self.Rp = rp, w, Rp
        c, u = self.commitment.getC(), self.commitment.getU()
        cp = (c * pow(u[0], rp, p)) % p
        dp = (d * pow(v0, rp, p)) % p
        ap = pow(a, w, p) * pow(g, Rp, p) % p
        
        bp_tmp = (b * pow(a0, rp, p)) % p
        bp_tmp2 = (c * pow(u[0], rp, p)) % p
        bp = pow(bp_tmp, w, p) * pow(bp_tmp2, Rp, p) % p

        ep = dumb_hash((g, h, cp, dp, ap, bp)) % q
        self.tmp = (ap, bp, cp, ep, dp)
        
        e = (ep * pow(w, q - 2, q)) % q
        return e
    
    def acceptProof(self, f: int):
        p, g, q = self.group.getPGQ()
        Rp, w = self.Rp, self.w
        fp = (Rp + w * f) % q
        ap, bp, cp, ep, dp = self.tmp
        self.tmp = None
        self.signature = (ap, bp, cp, dp, fp) # Useless to add ep the verifier can recompute it

    def getSignature(self):
        return self.signature

class CommitmentVerifier:
    def __init__(self, group: Group):
        self.group = group
    
    def checkSignature(self, h: int, signature: tuple):
        ap, bp, cp, dp, fp = signature
        p, g, q = group.getPGQ()
        ep = dumb_hash((g, h, cp, dp, ap, bp)) % q

        # Is the signature signed by the issuer owning the public key h ?
        check_signature = pow(g, fp, p) == (ap * pow(h, ep, p)) % p

        # Is commitment cp (re-randomized version) bind to this signature ?
        check_binding = pow(cp, fp, p) == (bp * pow(dp, ep, p)) % p

        return check_signature and check_binding

issuer = CommitmentIssuer(group)
prover = CommitmentProver(group, commitment)
verifier = CommitmentVerifier(group)
c, PI = prover.getCommitment()
t = issuer.getReCommitment(c, PI)
assert t != False
d, a, b, a0, v0 = t
e = prover.getChallenge(issuer.getPublicKey(), d, a, b, a0, v0)
f = issuer.getProof(e)
prover.acceptProof(f)
assert verifier.checkSignature(issuer.getPublicKey(), prover.getSignature())

In [139]:
def divertible_chaum_perderson_transcript():
    issuer = CommitmentIssuer(group)
    while True:
        issuer2 = CommitmentIssuer(group)
        if issuer2.x != issuer.x:
            break
    prover = CommitmentProver(group, commitment)
    prover2 = CommitmentProver(group, commitment)
    verifier = CommitmentVerifier(group)

    c, PI = prover.getCommitment()
    t = issuer.getReCommitment(c, PI)
    assert t != False
    d, a, b, a0, v0 = t
    e = prover.getChallenge(issuer.getPublicKey(), d, a, b, a0, v0)
    f = issuer.getProof(e)
    prover.acceptProof(f)

    c, PI = prover2.getCommitment()
    t = issuer2.getReCommitment(c, PI)
    assert t != False
    d, a, b, a0, v0 = t
    e = prover2.getChallenge(issuer2.getPublicKey(), d, a, b, a0, v0)
    f = issuer2.getProof(e)
    prover2.acceptProof(f)

    assert verifier.checkSignature(issuer.getPublicKey(), prover.getSignature())
    assert not verifier.checkSignature(issuer2.getPublicKey(), prover.getSignature())
    assert verifier.checkSignature(issuer2.getPublicKey(), prover2.getSignature())
    assert not verifier.checkSignature(issuer.getPublicKey(), prover2.getSignature())

for _ in range(100):
    divertible_chaum_perderson_transcript()